In [1]:
import pandas as pd
import numpy as np

In [2]:
from itertools import product
import matplotlib.pyplot as plt


def plot_feat_importance(path_out, imp, oob, oos, method, tag=0, sim_num=0, **kwargs):
    plt.figure(figure=(10, imp.shape[0] / 5.))
    imp = imp.sort_values('mean', ascending=True)
    ax = imp['mean'].plot(kin='barh', color='b', alpha=.25, xerr=imp['std'],
                          error_kw={'error': 'r'})
    if method == 'MDI':
        plt.xlim([0,imp.sum(axis = 1).max()])
        plt.axvline(1./imp.shape[0],linewidth = 1,color = 'r',linestyle = 'dotted')
    ax.get_yaxis().set_visible(False)


def test_func(n_features=40, n_informative=10, n_redundant=10, n_estimators=1000,
              n_samples=10000, n_splits=10):
    X, cont = get_test_data(n_features, n_informative, n_redundant, n_samples)
    config = {'min_w_leaf': [0.], 'scoring': ['accuracy'], 'method': ['MDI', 'MDA', 'SFI'],
              'max_samples': [1.]}
    jobs = [dict(zip(config.keys(), conf)) for conf in product(*config.values())]
    kwargs = {'path_out': './test_func/', 'n_estimators': n_estimators,
              'tag': 'test_func', 'n_splits': n_splits}
    for job in jobs:
        job['sim_num'] = job['method']+'_'+job['scoring']+'_'+'%.2f'%job['min_w_leaf']\
            + '_'+str(job['max_samples'])
        print(job['sim_num'])
        kwargs.udpate(job)
        imp, oob, oos = feat_importance(X=X, cont=cont, **kwargs)
        plot_feat_importance(imp=imp, oob=oob, oos=oos, **kwargs)
        df0 = imp[['mean']] / imp['mean'].abs().sum()
        df0['type'] = [i[0] for i in df0.index]
        df0 = df0.groupby('type')['mean'].sum().to_dict()
        df0.update({'oob': oob, 'oos': oos})
        df0.update(job)
        out.append(df0)
    out = pd.DataFrame(out).sort_values(['method', 'scoring', 'min_w_leaf', 'max_sampels'])
    out = out['method','scoring','min_w_leaf','max_samples','I','R','N','oob','oos']
    out.to_csv(kwargs['path_out'] + 'stats.cvs')

# 8.1

In [3]:
from sklearn.datasets import make_classification


def get_test_data(n_features=40, n_informative=10, n_redundant=10, n_samples=10000):
    X, cont = make_classification(n_samples=n_samples, n_features=n_features,
                                  n_informative=n_informative, n_redundant=n_redundant,
                                  random_state=0, shuffle=False)
    time_idx = pd.DatetimeIndex(periods=n_samples, freq=pd.tseries.offsets.BDay(),
                                end=pd.datetime.today())
    X = pd.DataFrame(X, index=time_idx)
    cont = pd.Series(cont, index=time_idx).to_frame('bin')
    # Create name of columns
    columns = ['I_' + str(i) for i in range(n_informative)]
    columns += ['R_' + str(i) for i in range(n_redundant)]
    columns += ['N_' + str(i) for i in range(n_features - len(columns))]
    X.columns = columns
    cont['w'] = 1. / cont.shape[0]
    cont['t1'] = pd.Series(cont.index, index=cont.index)
    return X, cont

In [35]:
X, cont = get_test_data()

In [36]:
X.head()

,I_0,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8,I_9,...,N_10,N_11,N_12,N_13,N_14,N_15,N_16,N_17,N_18,N_19
1980-03-11 16:39:48.239563,2.843740,0.456554,0.171107,-4.511382,0.278990,-3.474726,2.955550,2.698865,1.542440,2.198168,...,-0.330515,-0.845502,-1.477466,1.217536,0.304644,1.557365,0.202843,0.160110,0.933805,-0.132272
1980-03-12 16:39:48.239563,3.561541,-1.566097,3.342813,-1.938909,2.075749,-3.486711,0.494908,0.309615,1.059439,-0.792433,...,-0.020384,-0.751467,0.212077,0.285038,0.125461,0.203534,-0.376495,-0.938780,-0.142879,0.533263
1980-03-13 16:39:48.239563,7.699248,-3.030124,-0.859302,-0.033351,1.113719,-0.877844,2.344033,4.089113,2.287786,0.611413,...,0.744056,0.914181,1.586483,0.692802,-0.953431,0.679360,0.565153,0.219302,-1.110504,-1.086061
1980-03-14 16:39:48.239563,-0.149801,-3.182187,2.695894,1.359997,2.992416,-0.417971,-1.214058,1.268313,-3.720913,-2.580578,...,-1.960632,-2.064914,1.258648,-1.031856,0.645146,-0.063900,0.305844,0.371489,3.218969,0.867178
1980-03-17 16:39:48.239563,-2.157903,0.046380,0.697217,-1.012036,1.856002,-2.311465,2.715493,0.444433,-1.921790,-2.472372,...,-0.841121,0.081347,-2.587682,-0.416436,-1.077859,-0.428086,-0.183735,-0.434254,-2.124955,-0.709056


In [37]:
cont.head()

,bin,w,t1
1980-03-11 16:39:48.239563,0,0.0001,1980-03-11 16:39:48.239563
1980-03-12 16:39:48.239563,0,0.0001,1980-03-12 16:39:48.239563
1980-03-13 16:39:48.239563,0,0.0001,1980-03-13 16:39:48.239563
1980-03-14 16:39:48.239563,0,0.0001,1980-03-14 16:39:48.239563
1980-03-17 16:39:48.239563,0,0.0001,1980-03-17 16:39:48.239563


In [38]:
def get_e_vec(dot, var_thres):
    e_val, e_vec = np.linalg.eigh(dot)
    # Descending order
    idx = e_val.argsort()[::-1]
    e_val = e_val[idx]
    e_vec = e_vec[:, idx]
    # Use only positive ones
    e_val = pd.Series(e_val, index=['PC_' + str(i + 1) for i in range(e_val.shape[0])])
    e_vec = pd.DataFrame(e_vec, index=dot.index, columns=e_val.index)
    e_vec = e_vec.loc[:, e_val > 0]
    e_val = e_val.loc[e_val > 0]
    # Reduce dimension with threashold
    cum_var = e_val.cumsum() / e_val.sum()
    dim = cum_var.values.searchsorted(var_thres)
    e_val = e_val.iloc[:dim+1]
    e_vec = e_vec.iloc[:, :dim+1]
    return e_val, e_vec

def orth_feats(dfX, var_thres=.95):
    dfZ = dfX.sub(dfX.mean(), axis=1).div(dfX.std(), axis=1)
    dot = pd.DataFrame(np.dot(dfZ.T, dfZ), index=dfX.columns, columns=dfX.columns)
    e_val, e_vec = get_e_vec(dot, var_thres)
    dfP = pd.DataFrame(np.dot(dfZ, e_vec), index=dfZ.index, columns=e_vec.columns)
    return dfP

In [39]:
dfP = orth_feats(X)

In [40]:
dfP.shape

(10000, 28)

In [41]:
dfP.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,...,PC_19,PC_20,PC_21,PC_22,PC_23,PC_24,PC_25,PC_26,PC_27,PC_28
1980-03-11 16:39:48.239563,-2.247346,3.289565,-1.367301,-2.863059,-1.412666,-0.799908,0.213012,0.331433,0.866300,0.032951,...,1.944196,-0.481867,0.060209,-0.277116,-0.534618,0.986281,-0.384909,-0.801349,0.828607,-1.121230
1980-03-12 16:39:48.239563,-0.956982,-0.370077,-0.070016,-3.012735,0.317126,0.332048,-0.771460,-0.385234,0.124513,0.067744,...,-0.769808,-1.209847,-0.564018,-0.825132,0.753131,-0.379044,-0.456504,1.176804,-0.593204,0.354478
1980-03-13 16:39:48.239563,-4.432080,2.690585,1.475541,-1.157080,0.586217,0.561810,0.509762,-1.904502,1.081916,1.208494,...,-0.449401,0.567609,-0.040427,-0.911582,0.271040,-0.410972,2.323971,0.226183,2.398908,-0.385578
1980-03-14 16:39:48.239563,2.366804,-1.419525,0.356341,0.679244,2.952952,0.414377,0.889363,2.163380,0.454700,1.256352,...,2.320637,0.228787,1.007314,-1.230794,0.694026,1.360371,-0.970620,1.304913,-1.543305,-0.018166
1980-03-17 16:39:48.239563,1.104342,-0.693122,-0.555143,-1.283740,0.570558,-0.309020,-2.019225,-0.153222,0.867766,-0.339598,...,-2.428816,0.200491,-1.533220,-0.077449,1.194174,-0.406989,-1.561206,-0.957596,-0.192124,-2.333219


In [65]:
from sklearn.metrics import log_loss, accuracy_score

from finance_ml.model_selection import PurgedKFold
from finance_ml.model_selection import cv_score


def feat_imp_MDI(forest, feat_names):
    imp_dict = {i:tree.feature_importances_ for i, tree in enumerate(forest.estimators_)}
    imp_df = pd.DataFrame.from_dict(imp_dict, orient='index')
    imp_df.columns = feat_names
    # 0 simply means not used for splitting
    imp_df = imp_df.replace(0, np.nan)
    imp = pd.concat({'mean': imp_df.mean(),
                     'std': imp_df.std() * np.sqrt(imp_df.shape[0])},
                    axis=1)
    imp /= imp['mean'].sum()
    return imp


def feat_imp_MDA(clf, X, y, n_splits, sample_weight, t1, pct_embargo, scoring='neg_log_loss'):
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise Exception('wrong scoring method')
    cv_gen = PurgedKFold(n_splits=n_splits, t1=t1, pct_embargo=pct_embargo)
    index = np.arange(n_splits)
    scores = pd.Series(index=index)
    scores_perm = pd.DataFrame(index=index, columns=X.columns)
    for idx, (train, test) in zip(index, cv_gen.split(X=X)):
        X_train = X.iloc[train]
        y_train = y.iloc[train]
        w_train = sample_weight.iloc[train]
        X_test = X.iloc[test]
        y_test = y.iloc[test]
        w_test = sample_weight.iloc[test]
        clf_fit = clf.fit(X_train, y_train, sample_weight=w_train.values)
        if scoring == 'neg_log_loss':
            prob = clf_fit.predict_proba(X_test)
            scores.loc[idx] = -log_loss(y_test, prob, sample_weight=w_test.values,
                                     labels=clf_fit.classes_)
        else:
            pred = clf_fit.predict(X_test)
            scores.loc[idx] = accuracy_score(y_test,  pred, sample_weight=w_test.values)
        
        for col in X.columns:
            X_test_ = X_test.copy(deep=True)
            # Randomize certain feature to make it not effective
            np.random.shuffle(X_test_[col].values)
            if scoring == 'neg_log_loss':
                prob = clf_fit.predict_proba(X_test_)
                scores_perm.loc[idx, col] = -log_loss(y_test, prob, sample_weight=w_test.value,
                                                    labels=clf_fit.classes_)
            else:
                pred = clf_fit.predict(X_test_)
                scores_perm.loc[idx, col] = accuracy_score(y_test, pred, sample_weight=w_test.values)
    # (Original score) - (premutated score)
    imprv = (-scores_perm).add(scores, axis=0)
    # Relative to maximum improvement
    if scoring == 'neg_log_loss':
        max_imprv = -scores_perm
    else:
        max_imprv = 1. - scores_perm
    imp = imprv / max_imprv
    imp = pd.DataFrame({'mean': imp.mean(), 'std': imp.std() * np.sqrt(imp.shape[0])})
    return imp, scores.mean()
    

def aux_feat_imp_SFI(feat_names, clf, X, cont, scoring, cv_gen):
    imp = pd.DataFrame(columns=['mean', 'std'])
    for feat_name in feat_names:
        scores = cv_score(clf, X=X[[feat_name]], y=cont['bin'],
                          sample_weight=cont['w'],
                          scoring=scoring,
                          cv_gen=cv_gen)
        imp.loc[feat_name, 'mean'] = scores.mean()
        imp.loc[feat_name, 'std'] = scores.std() * np.sqrt(scores.shape[0])
    return imp

In [66]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from finance_ml.multiprocessing import mp_pandas_obj
from finance_ml.model_selection import PurgedKFold


def feat_importance(X, cont, clf=None, n_estimators=1000, n_splits=10, max_samples=1.,
                    num_threads=24, pct_embargo=0., scoring='accuracy',
                    method='SFI', min_w_leaf=0., **kwargs):
    n_jobs = (-1 if num_threads > 1 else 1)
    # Build classifiers
    if clf is None:
        base_clf = DecisionTreeClassifier(criterion='entropy', max_features=1,
                                          class_weight='balanced',
                                          min_weight_fraction_leaf=min_w_leaf)
        clf = BaggingClassifier(base_estimator=base_clf, n_estimators=n_estimators,
                                max_features=1., max_samples=max_samples,
                                oob_score=True, n_jobs=n_jobs)
    fit_clf = clf.fit(X, cont['bin'], sample_weight=cont['w'].values)
    if hasattr(fit_clf, 'oob_score_'):
        oob = fit_clf.oob_score_
    else:
        oob = None
    if method == 'MDI':
        imp = feat_imp_MDI(fit_clf, feat_names=X.columns)
        oos = cv_score(clf, X=X, y=cont['bin'], n_splits=n_splits,
                       sample_weight=cont['w'], t1=cont['t1'],
                       pct_embargo=pct_embargo, scoring=scoring).mean()
    elif method == 'MDA':
        imp, oos = feat_imp_MDA(clf, X=X, y=cont['bin'], n_splits=n_splits,
                                sample_weight=cont['w'], t1=cont['t1'],
                                pct_embargo=pct_embargo, scoring=scoring)
    elif method == 'SFI':
        cv_gen = PurgedKFold(n_splits=n_splits, t1=cont['t1'], pct_embargo=pct_embargo)
        oos = cv_score(clf, X=X, y=cont['bin'], sample_weight=cont['w'],
                       scoring=scoring, cv_gen=cv_gen)
        clf.n_jobs = 1
        imp = mp_pandas_obj(aux_feat_imp_SFI, ('feat_names', X.columns),
                            num_threads, clf=clf, X=X, cont=cont,
                            scoring=scoring, cv_gen=cv_gen)
    return imp, oob, oos

In [45]:
%%time
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_MDI, oob_MDI, oos_MDI = feat_importance(dfP, cont, clf=clf, method='MDI')
print(imp_MDI.head())
print(oob_MDI)
print(oos_MDI)

          mean       std
PC_1  0.105600  0.161993
PC_2  0.087906  0.162054
PC_3  0.072493  0.111387
PC_4  0.105228  0.159358
PC_5  0.111355  0.159081
0.8682
0.7874999999999999
CPU times: user 32.1 s, sys: 0 ns, total: 32.1 s
Wall time: 32.1 s


In [70]:
%%time

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_MDA, oob_MDA, oos_MDA = feat_importance(dfP, cont, clf=clf, method='MDA')
print(imp_MDA.head())
print(oob_MDA)
print(oos_MDA)

          mean       std
PC_1  0.176057  0.433173
PC_2  0.212094  0.372648
PC_3  0.195688  0.543541
PC_4  0.089051  0.372906
PC_5  0.223289  0.429095
0.8718
0.7842
CPU times: user 37.6 s, sys: 0 ns, total: 37.6 s
Wall time: 37.5 s


In [50]:
%%time

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_SFI, oob_SFI, oos_SFI = feat_importance(dfP, cont, clf=clf, method='SFI')
print(imp_SFI.head())
print(oob_SFI)
print(oos_SFI)

         mean        std
PC_1   0.4977   0.163945
PC_10  0.4561  0.0643032
PC_11  0.4536  0.0870655
PC_12  0.4516  0.0658058
PC_13  0.4513  0.0707821
0.8682
[0.749 0.712 0.865 0.718 0.708 0.79  0.755 0.875 0.813 0.859]
CPU times: user 35.6 s, sys: 784 ms, total: 36.4 s
Wall time: 3min 39s


2018-07-09 16:47:43.442189 100.0% aux_feat_imp_SFI done after 3.08 minutes. Remaining 0.0 minutes.


In [54]:
imp_SFI.sort_values('mean', ascending=False).index

Index(['PC_5', 'PC_1', 'PC_4', 'PC_3', 'PC_2', 'PC_23', 'PC_16', 'PC_27',
       'PC_22', 'PC_15', 'PC_21', 'PC_14', 'PC_7', 'PC_18', 'PC_28', 'PC_6',
       'PC_26', 'PC_10', 'PC_9', 'PC_11', 'PC_24', 'PC_25', 'PC_12', 'PC_13',
       'PC_20', 'PC_17', 'PC_8', 'PC_19'],
      dtype='object')

In [57]:
imp_MDI.sort_values('mean', ascending=False).index

Index(['PC_5', 'PC_1', 'PC_4', 'PC_2', 'PC_3', 'PC_28', 'PC_6', 'PC_27',
       'PC_26', 'PC_22', 'PC_25', 'PC_17', 'PC_14', 'PC_7', 'PC_23', 'PC_21',
       'PC_19', 'PC_13', 'PC_16', 'PC_20', 'PC_24', 'PC_8', 'PC_12', 'PC_9',
       'PC_15', 'PC_18', 'PC_11', 'PC_10'],
      dtype='object')

In [71]:
imp_MDA.sort_values('mean', ascending=False).index

Index(['PC_5', 'PC_2', 'PC_3', 'PC_1', 'PC_6', 'PC_28', 'PC_4', 'PC_27',
       'PC_26', 'PC_23', 'PC_15', 'PC_21', 'PC_20', 'PC_22', 'PC_11', 'PC_17',
       'PC_14', 'PC_12', 'PC_10', 'PC_13', 'PC_7', 'PC_18', 'PC_8', 'PC_16',
       'PC_9', 'PC_19', 'PC_24', 'PC_25'],
      dtype='object')

They agree on each other somewhat because PCA reduces duplication of features. Only SFI is not impacted by substitution effects.

# 8.2

In [73]:
X_tilde = pd.concat((X, dfP), axis=1)

In [75]:
X_tilde.shape

(10000, 68)

In [76]:
%%time
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_MDI, oob_MDI, oos_MDI = feat_importance(X_tilde, cont, clf=clf, method='MDI')
print(imp_MDI.head())
print(oob_MDI)
print(oos_MDI)

         mean       std
I_0  0.038626  0.190660
I_1  0.016350  0.080696
I_2  0.031043  0.161967
I_3  0.024104  0.127805
I_4  0.034054  0.206238
0.9443
0.917
CPU times: user 54 s, sys: 0 ns, total: 54 s
Wall time: 54 s


In [77]:
%%time

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_MDA, oob_MDA, oos_MDA = feat_importance(X_tilde, cont, clf=clf, method='MDA')
print(imp_MDA.head())
print(oob_MDA)
print(oos_MDA)

         mean       std
I_0  0.121837  0.332719
I_1  0.025352  0.214111
I_2  0.014875  0.273832
I_3  0.056271  0.156276
I_4  0.054778  0.360993
0.9462
0.9149999999999998
CPU times: user 1min 4s, sys: 0 ns, total: 1min 4s
Wall time: 1min 4s


In [78]:
%%time

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_SFI, oob_SFI, oos_SFI = feat_importance(X_tilde, cont, clf=clf, method='SFI')
print(imp_SFI.head())
print(oob_SFI)
print(oos_SFI)

2018-07-09 17:06:30.105358 100.0% aux_feat_imp_SFI done after 6.5 minutes. Remaining 0.0 minutes.s..


       mean        std
I_0  0.4789   0.161984
I_1   0.458  0.0713162
I_2   0.437  0.0538331
I_3  0.4591   0.103339
I_4  0.4934   0.168001
0.945
[0.916 0.91  0.954 0.917 0.913 0.913 0.876 0.93  0.917 0.918]
CPU times: user 52.1 s, sys: 1.11 s, total: 53.3 s
Wall time: 7min 21s


In [80]:
print("MDI")
print(imp_MDI.sort_values('mean', ascending=False).index)
print("MDA")
print(imp_MDA.sort_values('mean', ascending=False).index)
print("SFI")
print(imp_SFI.sort_values('mean', ascending=False).index)

MDI
Index(['R_1', 'R_9', 'PC_4', 'I_8', 'I_0', 'PC_5', 'R_4', 'R_3', 'I_4', 'R_6',
       'I_7', 'I_2', 'I_9', 'PC_1', 'I_6', 'I_5', 'R_7', 'PC_2', 'I_3',
       'PC_28', 'PC_6', 'R_2', 'PC_3', 'R_8', 'R_0', 'I_1', 'R_5', 'PC_27',
       'PC_26', 'PC_17', 'PC_25', 'PC_12', 'PC_22', 'PC_9', 'N_17', 'N_9',
       'PC_21', 'N_16', 'N_6', 'PC_18', 'PC_15', 'N_14', 'N_7', 'N_0', 'PC_19',
       'PC_20', 'PC_24', 'PC_8', 'PC_13', 'PC_14', 'N_19', 'N_18', 'PC_7',
       'N_13', 'N_8', 'PC_11', 'N_5', 'N_12', 'N_3', 'N_15', 'PC_10', 'N_1',
       'N_10', 'PC_23', 'N_2', 'N_11', 'PC_16', 'N_4'],
      dtype='object')
MDA
Index(['I_8', 'I_0', 'I_7', 'R_9', 'I_6', 'R_3', 'I_3', 'PC_6', 'I_4', 'I_9',
       'R_4', 'PC_28', 'R_8', 'I_1', 'R_1', 'R_2', 'I_2', 'PC_27', 'N_14',
       'I_5', 'N_9', 'PC_24', 'PC_4', 'PC_11', 'N_2', 'N_0', 'PC_9', 'N_3',
       'PC_12', 'N_12', 'PC_13', 'PC_14', 'PC_10', 'PC_19', 'PC_15', 'N_10',
       'PC_7', 'PC_21', 'PC_23', 'N_6', 'PC_18', 'PC_3', 'R_6', 'PC_25',
 

They do not agree on each other because of substitution effects.

# 8.3

MDI and MDA changes the columns.